In [13]:
import numpy as np
import faiss
import litellm
from litellm import completion
from dotenv import load_dotenv
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import ArxivLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Load API keys
load_dotenv(".env")
apikey = os.getenv('OPENAI_API_KEY')
print(apikey)

sk--lxq9ReUmWxgxJhWPPwRNg


In [14]:
import requests

# Example query to the arXiv API directly
url = "http://export.arxiv.org/api/query"
params = {
    "search_query": "all:Retrieval Augmented Generation",
    "start": 0,
    "max_results": 5
}

response = requests.get(url, params=params)

if response.status_code == 200:
    print("API Request Successful")
    print("Response:", response.text[:500])  # Print first 500 characters of the response
else:
    print("Failed to fetch data from arXiv:", response.status_code)

API Request Successful
Response: <?xml version="1.0" encoding="UTF-8"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <link href="http://arxiv.org/api/query?search_query%3Dall%3ARetrieval%20Augmented%20Generation%26id_list%3D%26start%3D0%26max_results%3D5" rel="self" type="application/atom+xml"/>
  <title type="html">ArXiv Query: search_query=all:Retrieval Augmented Generation&amp;id_list=&amp;start=0&amp;max_results=5</title>
  <id>http://arxiv.org/api/UOKvU43HASfzk48jVrFyBsmll1A</id>
  <updated>2024-05-23T00:00:00-04:00</updat


In [21]:
# Load the document pertaining to a particular topic
docs = ArxivLoader(query=""" all:"attention mechanisms" AND (all:"convolutional neural networks" OR all:"CNN") AND NOT all:"transformer" """, load_max_docs=5).load()

# Split the dpocument into smaller chunks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=350, chunk_overlap=50
)

chunked_documents = text_splitter.split_documents(docs)
print(chunked_documents)

[Document(page_content='1\nPulmonary Disease Classiﬁcation Using Globally\nCorrelated Maximum Likelihood:\nan Auxiliary Attention mechanism for\nConvolutional Neural Networks\nEdward Verenich, Tobias Martin, Alvaro Velasquez, Nazar Khan, and Faraz Hussain\nAbstract—Convolutional neural networks (CNN) are now being\nwidely used for classiﬁying and detecting pulmonary abnormal-\nities in chest radiographs. Two complementary generalization\nproperties of CNNs, translation invariance and equivariance,\nare particularly useful in detecting manifested abnormalities\nassociated with pulmonary disease, regardless of their spatial\nlocations within the image. However, these properties also come\nwith the loss of exact spatial information and global relative\npositions of abnormalities detected in local regions. Global\nrelative positions of such abnormalities may help distinguish\nsimilar conditions, such as COVID-19 and viral pneumonia. In\nsuch instances, a global attention mechanism is neede

In [16]:
print(type(docs))
print(len(docs))
print(type(docs[0].page_content))

print(type(chunked_documents))
print(len(chunked_documents))
print(chunked_documents[0])




<class 'list'>
5
<class 'str'>
<class 'list'>
199
page_content='1\nPulmonary Disease Classiﬁcation Using Globally\nCorrelated Maximum Likelihood:\nan Auxiliary Attention mechanism for\nConvolutional Neural Networks\nEdward Verenich, Tobias Martin, Alvaro Velasquez, Nazar Khan, and Faraz Hussain\nAbstract—Convolutional neural networks (CNN) are now being\nwidely used for classiﬁying and detecting pulmonary abnormal-\nities in chest radiographs. Two complementary generalization\nproperties of CNNs, translation invariance and equivariance,\nare particularly useful in detecting manifested abnormalities\nassociated with pulmonary disease, regardless of their spatial\nlocations within the image. However, these properties also come\nwith the loss of exact spatial information and global relative\npositions of abnormalities detected in local regions. Global\nrelative positions of such abnormalities may help distinguish\nsimilar conditions, such as COVID-19 and viral pneumonia. In\nsuch instance

In [23]:
# Test
response = completion(
    api_key=apikey,
    base_url="https://drchat.xyz",
    model = "gpt-3.5-turbo-16k",
    custom_llm_provider="openai",
    messages = [{ "content": "What is a cat?","role": "user"}],
    temperature=0.5
)
print(response.choices[0].message.content)

A cat is a small, carnivorous mammal that is often kept as a pet. They are known for their independent nature, agility, and hunting abilities. Cats have sharp retractable claws, keen senses, and a flexible body, which allows them to climb and jump with ease. They come in various breeds, sizes, colors, and coat patterns. Cats are known for their grooming habits, as they clean themselves by licking their fur. They communicate through a combination of vocalizations, body language, and scent marking. Cats are popular pets due to their companionship, playful nature, and ability to provide comfort to their owners.


In [35]:
user_query = input("Ask a research question!")

# Create multiple search queries
search_split_prompt = f"""
Your role is that of a researcher attempting to answer a question. Given a question from the user,
your job is to come up with one of more ArXiv queries that searches for the exact information needed to answer the question.
A single question may need multiple queries. If there are multiple queries, make sure they are separated by "|"

You can include all syntax that involves including multiple terms, search by abstract, title, etc.

Example 1:
Given question: What are the ethical concerns associated with the use of facial recognition technology?
Your Answer: ("facial recognition technology" OR "facial recognition systems" OR "facial recognition software") AND ("ethical concerns" OR "ethical implications" OR "ethical issues")

Example 2:
Given question: What are some prominent attention mechanisms for convolutional neural networks, and how are they used in the autonomous vehicle industry?
Your Answer: all:"attention mechanisms" AND ("convolutional neural networks" OR "CNN") | all:"attention mechanisms" AND ("autonomous vehicles" OR "self-driving cars")

Question: {user_query},

As shown above, your response should be formatted as follows:

ArXiv Query 1 | ArXiv Query 2 | ArXiv Query 3 | ...

"""
response = completion(
    api_key=apikey,
    base_url="https://drchat.xyz",
    model = "gpt4-1106-preview",
    custom_llm_provider="openai",
    messages = [{ "content": search_split_prompt,"role": "user"}],
    temperature=0.5
)
print("Response recieved")
print(response.choices[0].message.content)
arxiv_queries_list = response.choices[0].message.content.split("|")


# Each element contains vector stores for each search query developed by LLM
chunks_for_queries = []
for q in arxiv_queries_list:
    docs = ArxivLoader(query=q, load_max_docs=5).load()
    print(docs)
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=350, chunk_overlap=50
    )
    chunked_documents = text_splitter.split_documents(docs)
    chunks_for_queries.append(chunked_documents)

print(type(chunks_for_queries[0][0].page_content))
print(chunks_for_queries[0][1].page_content)

Response recieved
("CBAM" OR "Convolutional Block Attention Module") AND "attention in CNNs" | "Convolutional Block Attention Module" AND "mechanism" | "CBAM" AND "convolutional neural networks"
[Document(page_content='1\nLearning to ignore: rethinking attention in CNNs\nFiras Laakom*, Kateryna Chumachenko*, Jenni Raitoharju, Alexandros Iosiﬁdis, and Moncef Gabbouj\nAbstract—Recently, there has been an increasing interest in applying attention mechanisms in Convolutional Neural Networks (CNNs)\nto solve computer vision tasks. Most of these methods learn to explicitly identify and highlight relevant parts of the scene and pass the\nattended image to further layers of the network. In this paper, we argue that such an approach might not be optimal. Arguably, explicitly\nlearning which parts of the image are relevant is typically harder than learning which parts of the image are less relevant and, thus,\nshould be ignored. In fact, in vision domain, there are many easy-to-identify patterns

In [36]:
print(type(chunks_for_queries))
print(len(chunks_for_queries[0]))

<class 'list'>
36


In [37]:
# Instantiate the Embedding Model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",openai_api_key=apikey, base_url="https://drchat.xyz")
# Create Index- Load document chunks into the vectorstore
vectorstore_list = []
for x in chunks_for_queries:
    faiss_vectorstore = FAISS.from_documents(
        documents=x,
        embedding=embeddings,
    )
    print(type(faiss_vectorstore))
    vectorstore_list.append(faiss_vectorstore)


# Create a retriver and retrieve relevant documents for each vector store
relevant_documents_list = []
for x in vectorstore_list:
    
    relevant_documents = x.similarity_search(user_query, k = 5)
    print(type(relevant_documents))
    relevant_documents_list.append(relevant_documents)

<class 'langchain_community.vectorstores.faiss.FAISS'>
<class 'langchain_community.vectorstores.faiss.FAISS'>
<class 'langchain_community.vectorstores.faiss.FAISS'>
<class 'list'>
<class 'list'>
<class 'list'>


In [38]:
print(relevant_documents_list)

[[Document(page_content='different attention maps compared to the explicit attention,\ni.e., learning to attend. Noticeably, standard CBAM attention\ntries to capture the relevant parts of the image directly,\nleading to the prediction being made based on the small\npart of the input that is considered by the model as the most\nimportant. This leads to the possibility that the model can\nmiss some important parts of the class of interest on the\nimage. As an example, only one of the plants on the lower\n6\nMMTM\nIgn1(α=1)\nIgn1(α=0.5)\nIgn1(α=0.8)\nIgn2\nIgn3\nNTU-RGBD\n89.98\n89.99\n90.52\n88.70\n90.21\n90.36\nTABLE 4\nAccuracy on NTU-RGBD dataset\nFig. 1. Validation loss curves of ResNet50 on CIFAR100 using the different attention approaches.\nﬁgure is considered in CBAM model, as well as only a side\nof the bus in the middle image. On the other hand, our ap-\nproach by learning to identify the non-relevant background\nregions ﬁrst and subsequently suppressing them, simpliﬁes\nthe pr

In [39]:
question_prompt = f"""
Given the following context: {relevant_documents_list}

Answer the following question: {user_query}

Only answer the question if the answer is in the context. Otherwise, say that you don't know.
"""

response = completion(
    api_key=apikey,
    base_url="https://drchat.xyz",
    model = "gpt-3.5-turbo-16k",
    custom_llm_provider="openai",
    messages = [{ "content": question_prompt,"role": "user"}],
    temperature=0.5
)

print(response.choices[0].message.content)

CBAM (Convolutional Block Attention Module) sequentially infers attention maps along two separate dimensions, channel and spatial. It first applies global average pooling and global maximum pooling operations along the channel axis to obtain overall information and spatial correlations, respectively. These operations generate average samples and max samples, which are then concatenated and passed through a convolutional layer to generate the spatial attention maps. The channel attention module emphasizes inter-channel associations by focusing on the most significant features across various channels. The spatial attention module focuses on crucial spatial locations by considering the spatial correlations between features. The attention maps are multiplied with the input feature map for adaptive feature refinement, resulting in a final refined output. CBAM can be integrated into any CNN architecture with negligible overhead and is end-to-end trainable along with the base CNNs.
